In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import re
import os


os.listdir('data_down')

['inv_7.csv']

In [3]:
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4,18,19,20,39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [59]:
df.columns

Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
       'wms_warehouse_name', 'lpn', 'sku_code', 'sku_name', 'sku_desc',
       'product_status', 'batch_number', 'inventory_status', 'location',
       'location_zone', 'location_class', 'location_type', 'location_status',
       'container_id', 'product_group', 'product_type', 'allocated_qty',
       'on_hand_qty', 'in_transit_qty', 'shelf_days', 'recived_date',
       'lock_codes', 'create_date', 'last_modified_date', 'data_source',
       'manufactured_date', 'expiration_date', 'ou_code', 'usage_flag',
       'inc_day', 'src'],
      dtype='object')

In [60]:
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))

In [63]:
df[time_cols].isna().sum()

recived_date             139
create_date              139
last_modified_date         0
manufactured_date     481935
expiration_date        10040
dtype: int64

In [75]:
df[df['ou_code'].isna()]

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,lpn,sku_code,sku_name,sku_desc,product_status,batch_number,...,lock_codes,create_date,last_modified_date,data_source,manufactured_date,expiration_date,ou_code,usage_flag,inc_day,src
0,Michelin,Michelin,NaN,NaN,NaN,451092_101,NaN,NaN,ONHAND_ES-1HB,2124N0,...,ONHAND_ES-1HB,2021-09-06,2021-09-25,scale_michelin,NaN,2024-06-12,NaN,1,20211027,scale
1,Michelin,Michelin,NaN,NaN,NaN,528570_101,NaN,NaN,ONHAND_ES-1HB,2112N0,...,ONHAND_ES-1HB,2021-09-17,2021-10-03,scale_michelin,NaN,2024-03-20,NaN,1,20211027,scale
2,Michelin,Michelin,NaN,NaN,NaN,528570_101,NaN,NaN,ONHAND_ES-1HB,2111N0,...,ONHAND_ES-1HB,2021-09-15,2021-10-03,scale_michelin,NaN,2024-03-13,NaN,1,20211027,scale
3,Michelin,Michelin,NaN,NaN,NaN,406040_112,NaN,NaN,ONHAND_US-1BF,2101N0,...,ONHAND_US-1BF,2021-09-27,2021-09-27,scale_michelin,NaN,2024-01-03,NaN,1,20211027,scale
4,Michelin,Michelin,NaN,NaN,NaN,745434_104,NaN,NaN,ONHAND_CN-17V,2029N0,...,ONHAND_CN-17V,2021-10-09,2021-10-09,scale_michelin,NaN,2023-07-20,NaN,1,20211027,scale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492830,Solvay-SHANGHAI,Solvay-SHANGHAI,FAS,FAS,NaN,118162,NaN,NaN,Bonded Normal,210105070831,...,Bonded Normal,2021-01-06,2021-01-08,scale_fas,NaN,4712-12-31,NaN,1,20211103,scale
492831,Solvay-SHANGHAI,Solvay-SHANGHAI,FAS,FAS,NaN,150085,NaN,NaN,Bonded Normal,201009101951,...,Bonded Normal,2020-10-10,2021-01-07,scale_fas,NaN,4712-12-31,NaN,1,20211103,scale
492832,Solvay-SHANGHAI,Solvay-SHANGHAI,FAS,FAS,NaN,64103,NaN,NaN,Bonded Normal,201226005700,...,Bonded Normal,2020-12-28,2021-01-08,scale_fas,NaN,4712-12-31,NaN,1,20211103,scale
492833,Solvay-SHANGHAI,Solvay-SHANGHAI,FAS,FAS,NaN,149875,NaN,NaN,Bonded Normal,201226002536,...,Bonded Normal,2020-12-29,2021-01-08,scale_fas,NaN,4712-12-31,NaN,1,20211103,scale
